# Imports and program variables


In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D, MaxPooling1D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
import pandas as pd

from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, Conv1D
from keras.callbacks import ReduceLROnPlateau,ModelCheckpoint

from sklearn.model_selection import train_test_split # to split dataset into train and test
from sklearn import preprocessing
from keras.preprocessing.sequence import pad_sequences

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from sklearn import preprocessing

import sys
sys.path.append('../')
from time_series_dataset_loader import TimeSeriesDatasetLoader


Using TensorFlow backend.
/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

In [2]:
import os
os.listdir('../../vote_daset.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../../vote_daset.csv'

In [ ]:
def linearize_dataset(dataset):
    ''' Returns a 1 dimensional representation from the dataset '''
    new_dataset = []

    for data_instance in dataset:
        new_dataset_cell = []
        for t_instance in data_instance:
            for value in t_instance[2:]:
                new_dataset_cell.append(value)

        new_dataset.append(new_dataset_cell)
    return np.asarray(new_dataset)
    
    

In [ ]:
dataset_path = '../../vote_daset.csv'

In [ ]:
def preprocessing_data(X, y):
    X = np.asarray(X)
    y = np.asarray(y)
    
    max_len = len(X[0])
    for row in X:
        if len(row) > max_len:
            max_len = len(row)

    X = pad_sequences(X, maxlen=max_len, padding='post', dtype='float64')
    X = linearize_dataset(X)
    return X,y
    

In [ ]:
def get_model():
#     model = Sequential()
#     model.add(Conv1D(128, 5,padding='same', input_shape=(504, 504)))
#     model.add(Activation('sigmoid'))
#     model.add(Conv1D(64, 5,padding='same'))
#     model.add(Activation('sigmoid'))
#     model.add(Dropout(0.2))
#     model.add(MaxPooling1D(pool_size=(8)))
#     model.add(Conv1D(32, 5,padding='same',))
#     model.add(Activation('sigmoid'))

#     model.add(Flatten())
#     model.add(Dense(3))
#     model.add(Activation('softmax'))
    model = Sequential()
    model.add(Embedding(1, 504))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3, activation='sigmoid'))
    
    return model

In [ ]:
a = np.full(100, 2)
b = np.full(2, 100)

In [ ]:
POSITIVE_VALUES = np.where(y==1)[0]
NEUTRAL_VALUES = np.where(y==0)[0]
NEGATIVE_VALUES = np.where(y==2)[0][0:600]

In [ ]:
def balance_dataset(X, y):
    X = np.asarray(X)
    y = np.asarray(y)
    POSITIVE_VALUES = np.where(y==1)[0]
    NEUTRAL_VALUES = np.where(y==0)[0]
    NEGATIVE_VALUES = np.where(y==2)[0][0:600]
    all_indexes = POSITIVE_VALUES.tolist() + NEUTRAL_VALUES.tolist() + NEGATIVE_VALUES.tolist()
    new_y = np.full(len(POSITIVE_VALUES), 1).tolist() \
        + np.full(len(NEUTRAL_VALUES), 0).tolist() \
        + np.full(len(NEGATIVE_VALUES), 2).tolist()
    
    
    return np.take(X, all_indexes), new_y
    
    
    

In [ ]:
np.take(X, all_indexes)

In [ ]:
os.listdir('../..')

In [3]:
DATASET_PATH = '../../vote_dataset.csv'


## Setting up train and test datasets

In [4]:
dataframe = pd.read_csv(DATASET_PATH, index_col=0)

In [5]:
X = dataframe[['0', '1', '2', '3', '4', '5', '6']].values

In [6]:
y = dataframe['7'].values


In [26]:
y

array([5, 2, 0, ..., 0, 6, 3])

In [18]:
binary_X = []
for instx in X:
    binary_X.append([val for val in map(lambda x: 1 if x>0.5 else 0, instx)])
    

In [19]:
binary_X

[[0, 0, 0, 1, 1, 1, 0],
 [0, 1, 0, 1, 0, 0, 1],
 [1, 1, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 0, 0],
 [1, 1, 0, 0, 1, 0, 0],
 [1, 1, 1, 0, 0, 0, 1],
 [0, 1, 0, 1, 0, 1, 0],
 [0, 0, 0, 1, 0, 0, 0],
 [0, 0, 1, 1, 0, 0, 0],
 [1, 0, 1, 0, 1, 0, 0],
 [0, 0, 0, 1, 1, 1, 0],
 [0, 1, 0, 1, 0, 0, 1],
 [0, 1, 0, 1, 0, 0, 1],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 1, 1, 1, 0],
 [0, 0, 1, 1, 1, 1, 0],
 [1, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 1, 0],
 [0, 0, 0, 1, 0, 1, 0],
 [0, 0, 0, 1, 1, 1, 0],
 [0, 0, 0, 1, 1, 1, 0],
 [0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 1, 1, 0],
 [0, 1, 1, 0, 0, 0, 1],
 [1, 1, 1, 0, 0, 0, 1],
 [0, 0, 0, 1, 1, 1, 0],
 [0, 0, 0, 1, 1, 1, 0],
 [1, 1, 0, 0, 1, 0, 0],
 [1, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 1, 0],
 [1, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 1, 0],
 [1, 1, 0, 0, 1, 0, 0],
 [0, 0, 1, 1, 1, 1, 0],
 [1, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 1, 1, 0],
 [0, 0, 0, 1, 1, 0, 0],
 [0, 1, 0, 1, 1, 0, 0],
 [1, 0, 0, 0, 1,

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)


In [8]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [79]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

def run_classifier(classifier, params, X, y):
    clf = GridSearchCV(classifier, params)
    clf.fit(X, y)
#     clf.transform(X)
    return pd.DataFrame(clf.cv_results_)

In [28]:
y_labels = y

In [64]:
## Linear Regression ##

from sklearn import linear_model
classifier = linear_model.LinearRegression()
params = {}
# run_classifier(clf, params, X, to_categorical(y))

##########################


In [55]:
y

array([5, 2, 0, ..., 0, 6, 3])

In [75]:
### LDA ####
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
clf = LinearDiscriminantAnalysis()
params = {'solver': ['svd', 'lsqr', 'eigen']} 
run_classifier(clf, params, X, y)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003890,0.000596,0.000725,0.000240,svd,{'solver': 'svd'},0.287554,0.317597,0.27897,0.291845,0.331897,0.301572,0.019891,1
1,0.006782,0.002962,0.001145,0.000435,lsqr,{'solver': 'lsqr'},0.287554,0.317597,0.27897,0.291845,0.331897,0.301572,0.019891,1
2,0.007504,0.002470,0.001119,0.000714,eigen,{'solver': 'eigen'},0.287554,0.317597,0.27897,0.291845,0.331897,0.301572,0.019891,1


In [81]:
### SVM ###
from sklearn.svm import SVC
params = {'kernel':['linear', 'poly', 'rbf', 'sigmoid'], 'C':[1, 2, 4,5, 6, 8, 10]}
svc = svm.SVC()
run_classifier(svc, params,X, y)

###########

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.035000,0.004433,0.005815,0.001282,1,linear,"{'C': 1, 'kernel': 'linear'}",0.296137,0.248927,0.266094,0.283262,0.271552,0.273194,0.015936,21
1,0.533699,0.058332,0.007199,0.000941,1,poly,"{'C': 1, 'kernel': 'poly'}",0.321888,0.334764,0.360515,0.291845,0.344828,0.330768,0.023205,7
2,0.049277,0.002601,0.008504,0.001260,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.330472,0.317597,0.296137,0.339056,0.323276,0.321308,0.014493,13
3,0.104571,0.005356,0.015046,0.001674,1,sigmoid,"{'C': 1, 'kernel': 'sigmoid'}",0.145923,0.137339,0.141631,0.141631,0.185345,0.150374,0.017695,22
4,0.033715,0.001987,0.006013,0.001080,2,linear,"{'C': 2, 'kernel': 'linear'}",0.287554,0.261803,0.283262,0.253219,0.293103,0.275788,0.015486,20
5,1.103830,0.108768,0.006635,0.001061,2,poly,"{'C': 2, 'kernel': 'poly'}",0.330472,0.334764,0.356223,0.300429,0.357759,0.335929,0.020879,4
6,0.055323,0.002872,0.008697,0.001151,2,rbf,"{'C': 2, 'kernel': 'rbf'}",0.330472,0.317597,0.296137,0.330472,0.336207,0.322177,0.014377,12
7,0.107927,0.004644,0.015555,0.001673,2,sigmoid,"{'C': 2, 'kernel': 'sigmoid'}",0.145923,0.137339,0.141631,0.141631,0.185345,0.150374,0.017695,22
8,0.037819,0.002624,0.006433,0.000792,4,linear,"{'C': 4, 'kernel': 'linear'}",0.300429,0.278970,0.287554,0.253219,0.301724,0.284379,0.017713,19
9,2.123775,0.126752,0.005666,0.001152,4,poly,"{'C': 4, 'kernel': 'poly'}",0.321888,0.317597,0.343348,0.317597,0.375000,0.335086,0.022124,5


In [77]:
### LDA ###
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
params = {'solver':['svd', 'lsqr', 'eigen'], 'shrinkage':[None, 'auto']}
clf = LinearDiscriminantAnalysis()
run_classifier(clf, params, )

###########

/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
NotImplementedError: shrinkage not supported

  FitFailedWarning)
/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
NotImplementedError: shrinkage not supported

  FitFailedWarning)
/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
NotImplementedError: shrinkage not supported

  FitFailedWarning)
/home/gabriel/.pyenv/versions/machine-learning/lib/pyth

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_shrinkage,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003832,0.001135,0.000810,0.000289,None,svd,"{'shrinkage': None, 'solver': 'svd'}",0.287554,0.317597,0.278970,0.291845,0.331897,0.301572,0.019891,1
1,0.004070,0.001029,0.000833,0.000468,None,lsqr,"{'shrinkage': None, 'solver': 'lsqr'}",0.287554,0.317597,0.278970,0.291845,0.331897,0.301572,0.019891,1
2,0.004455,0.001658,0.000548,0.000164,None,eigen,"{'shrinkage': None, 'solver': 'eigen'}",0.287554,0.317597,0.278970,0.291845,0.331897,0.301572,0.019891,1
3,0.000967,0.000155,0.000000,0.000000,auto,svd,"{'shrinkage': 'auto', 'solver': 'svd'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
4,0.011783,0.001667,0.000741,0.000338,auto,lsqr,"{'shrinkage': 'auto', 'solver': 'lsqr'}",0.287554,0.291845,0.270386,0.287554,0.349138,0.297295,0.026949,4
5,0.013245,0.000364,0.000744,0.000197,auto,eigen,"{'shrinkage': 'auto', 'solver': 'eigen'}",0.287554,0.291845,0.270386,0.287554,0.349138,0.297295,0.026949,4


In [80]:
### KNN ###
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
params = {'n_neighbors':[3,5,7,10,50,100], 'weights':['uniform', 'distance'], 
         'algorithm':['auto', 'ball_tree', 'kd_tree', 'brute']}
clf = KNeighborsClassifier()
run_classifier(clf, params, X, y)

###########

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_algorithm,param_n_neighbors,param_weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002100,0.000582,0.022827,0.004043,auto,3,uniform,"{'algorithm': 'auto', 'n_neighbors': 3, 'weigh...",0.287554,0.270386,0.270386,0.223176,0.275862,0.265473,0.022058,45
1,0.001633,0.000557,0.003875,0.001204,auto,3,distance,"{'algorithm': 'auto', 'n_neighbors': 3, 'weigh...",0.309013,0.274678,0.309013,0.296137,0.271552,0.292079,0.016212,17
2,0.001514,0.000465,0.015342,0.000689,auto,5,uniform,"{'algorithm': 'auto', 'n_neighbors': 5, 'weigh...",0.326180,0.270386,0.257511,0.244635,0.306034,0.280949,0.030512,33
3,0.001476,0.000352,0.003824,0.000926,auto,5,distance,"{'algorithm': 'auto', 'n_neighbors': 5, 'weigh...",0.313305,0.278970,0.296137,0.257511,0.275862,0.284357,0.018967,29
4,0.001474,0.000258,0.016133,0.002052,auto,7,uniform,"{'algorithm': 'auto', 'n_neighbors': 7, 'weigh...",0.313305,0.274678,0.257511,0.274678,0.275862,0.279207,0.018361,37
5,0.001438,0.000481,0.004006,0.000937,auto,7,distance,"{'algorithm': 'auto', 'n_neighbors': 7, 'weigh...",0.330472,0.278970,0.287554,0.253219,0.288793,0.287802,0.024883,25
6,0.001647,0.000535,0.018849,0.002820,auto,10,uniform,"{'algorithm': 'auto', 'n_neighbors': 10, 'weig...",0.321888,0.278970,0.253219,0.244635,0.250000,0.269742,0.028627,41
7,0.001200,0.000085,0.003962,0.000461,auto,10,distance,"{'algorithm': 'auto', 'n_neighbors': 10, 'weig...",0.356223,0.270386,0.283262,0.274678,0.293103,0.295531,0.031328,9
8,0.001315,0.000307,0.020377,0.001917,auto,50,uniform,"{'algorithm': 'auto', 'n_neighbors': 50, 'weig...",0.313305,0.283262,0.296137,0.283262,0.267241,0.288641,0.015364,21
9,0.001397,0.000275,0.007641,0.000764,auto,50,distance,"{'algorithm': 'auto', 'n_neighbors': 50, 'weig...",0.321888,0.309013,0.309013,0.287554,0.293103,0.304114,0.012324,5


In [ ]:
### Naive Bayes ###
import numpy as np
from sklearn.naive_bayes import GaussianNB
params = {}
clf = GaussianNB()
run_classifier(clf, params, X_train, X_test, y_train, y_test)

###########

In [ ]:
### Decision Tree ###
import numpy as np
from sklearn.tree import DecisionTreeClassifier
params = {'criterion': ['gini', 'entropy'], 'splitter': ['best', 'random'], 'min_samples_split':[2,4,6,10,20,50,100],
         'min_samples_leaf':[1,2,5,10,20,50,100]}
clf = DecisionTreeClassifier()
run_classifier(clf, params, X_train, X_test, y_train, y_test)

###########

In [83]:
### MLP classifier ###

import numpy as np
from sklearn.neural_network import MLPClassifier
params = {
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [0.0001, 0.0005, 0.0020, 0.0050, 0., ] ,
    'max_iter': [1000]
}

clf = MLPClassifier()
run_classifier(clf, params, X, y)

#####################

/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_pe

/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_pe

/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_pe

/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: Convergenc

/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/gabriel/.pyenv/versions/machine-learning/lib/python3.6/site-packages/sklearn/neural_network/_multilayer_pe

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_activation,param_alpha,param_max_iter,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,4.099216,0.254667,0.001404,0.000194,identity,0.0001,1000,lbfgs,"{'activation': 'identity', 'alpha': 0.0001, 'm...",0.300429,0.326180,0.278970,0.309013,0.344828,0.311884,0.022416,20
1,2.805539,0.208784,0.001163,0.000336,identity,0.0001,1000,sgd,"{'activation': 'identity', 'alpha': 0.0001, 'm...",0.266094,0.261803,0.274678,0.261803,0.284483,0.269772,0.008730,39
2,2.320917,0.346581,0.000889,0.000136,identity,0.0001,1000,adam,"{'activation': 'identity', 'alpha': 0.0001, 'm...",0.270386,0.283262,0.287554,0.283262,0.323276,0.289548,0.017820,34
3,4.093038,0.869015,0.002092,0.001009,identity,0.0005,1000,lbfgs,"{'activation': 'identity', 'alpha': 0.0005, 'm...",0.300429,0.326180,0.274678,0.313305,0.344828,0.311884,0.023695,20
4,2.776346,0.199324,0.000848,0.000207,identity,0.0005,1000,sgd,"{'activation': 'identity', 'alpha': 0.0005, 'm...",0.283262,0.240343,0.261803,0.278970,0.280172,0.268910,0.016128,40
5,2.402398,0.378572,0.001214,0.000305,identity,0.0005,1000,adam,"{'activation': 'identity', 'alpha': 0.0005, 'm...",0.300429,0.270386,0.287554,0.278970,0.336207,0.294709,0.023001,31
6,4.269989,0.341112,0.006250,0.005964,identity,0.002,1000,lbfgs,"{'activation': 'identity', 'alpha': 0.002, 'ma...",0.300429,0.330472,0.270386,0.309013,0.344828,0.311026,0.025646,23
7,2.919401,0.388047,0.001160,0.000568,identity,0.002,1000,sgd,"{'activation': 'identity', 'alpha': 0.002, 'ma...",0.278970,0.240343,0.287554,0.266094,0.267241,0.268041,0.015949,43
8,2.582799,0.709713,0.000971,0.000495,identity,0.002,1000,adam,"{'activation': 'identity', 'alpha': 0.002, 'ma...",0.283262,0.257511,0.287554,0.287554,0.331897,0.289555,0.023947,33
9,3.981872,0.222343,0.001673,0.000435,identity,0.005,1000,lbfgs,"{'activation': 'identity', 'alpha': 0.005, 'ma...",0.300429,0.334764,0.274678,0.309013,0.344828,0.312742,0.025015,19


In [ ]:
X_traincnn = np.expand_dims(X_train, axis=0)
X_testcnn = np.expand_dims(X_test, axis=0)

In [ ]:
X_train = X_train.reshape(1, len(X_train), 504)
X_test = X_test.reshape(1, len(X_test), 504)

In [ ]:
embedding_size  = 128
kernel_size = 5
filters = 64
pool_size = 4

lstm_output_size = 70

# Training
batch_size = 64
epochs = 100

model = Sequential()
model.add(Embedding(1, embedding_size, input_length=504))
model.add(Dropout(0.25))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
model.add(MaxPooling1D(pool_size=pool_size))
model.add(LSTM(lstm_output_size))
model.add(Dense(3))
model.add(Activation('sigmoid'))


from keras.optimizers import SGD
opt = SGD(lr=0.01)
model.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy'])
cnnhistory=model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(X_test, y_test))



In [ ]:
batch_size = 256
epochs = 500
from keras.optimizers import SGD
opt = SGD(lr=0.01)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = opt, metrics=['accuracy'])
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=20, min_lr=0.0000001)
mcp_save = ModelCheckpoint('../model_checkpoints/two_split.h5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
cnnhistory=model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(X_test, y_test))
# , callbacks=[mcp_save, lr_reduce]

In [ ]:
model.summary()

In [ ]:
''' More Preprocessing '''

In [ ]:
from sklearn.preprocessing import RobustScaler

scalers = {}
for i in range(X.shape[1]):
    scalers[i] = RobustScaler()
    X[:, i, :] = scalers[i].fit_transform(X[:, i, :]) 


In [ ]:
from sklearn.preprocessing import Normalizer

scalers = {}
for i in range(X.shape[1]):
    scalers[i] = Normalizer()
    X[:, i, :] = scalers[i].fit_transform(X[:, i, :]) 

In [ ]:
X[0][0]

In [ ]:
X_train.reshape(1)

In [ ]:
''' Finishing Preprocessing '''

In [ ]:
# See X and y details
print(X[:2])
print(X.shape)

print(y[:3])
print(y.shape)

In [ ]:
# Split the dataset in train and test

print("\nX_train:\n")
print(X_train[0][0])
print(X_train.shape)

print("\nX_test:\n")
print(X_test[0][0])
print(X_test.shape)

print("\ny_train:\n")
print(y_train[:3])
print(y_train.shape)

print("\ny_test:\n")
print(y_test[:3])
print(y_test.shape)

In [ ]:
# Create categorical matrices



In [ ]:
num_rows = X[0].shape[0]
num_columns = X[0].shape[1]
num_channels = 1

X_train = X_train.reshape(X_train.shape[0], num_rows, num_columns, num_channels)
X_test = X_test.reshape(X_test.shape[0], num_rows, num_columns, num_channels)

print("\nX_train:\n")
print(X_train[0][0])
print(X_train.shape)

print("\nX_test:\n")
print(X_test[0][0])
print(X_test.shape)

## Building and Compiling Model

In [ ]:
# top-k category accuracy
import functools
import keras
top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'

# compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',top3_acc])

In [ ]:
# Define bath and epochs
batch_size = 256
epochs = 400
checkpoint_file = '../models/model_checkpoints/original_window_2.h5'
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',top3_acc])
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=20, min_lr=0.0000001)
mcp_save = ModelCheckpoint('../models/model_checkpoints/two_split.h5', save_best_only=True, monitor='val_loss', mode='min')
cnnhistory=model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(X_test, y_test), callbacks=[mcp_save, lr_reduce])

In [ ]:
accuracy_list = cnnhistory.history['val_accuracy']
highest_index = cnnhistory.history['val_accuracy'].index(np.sort(cnnhistory.history['val_accuracy'])[-1])
print(cnnhistory.history['val_accuracy'][highest_index])

In [ ]:
# without prepocessing: 0.5863248109817505
# robust preprocessing: 0.6034188270568848
# normalization: 0.6239316463470459

In [ ]:
import matplotlib.pyplot as plt

# Model Summary

model.summary()

# PLT History info
plt.plot(cnnhistory.history['loss'])
plt.plot(cnnhistory.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()